## ライブラリの定義

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 例外処理用のライブラリをインポート
from selenium.common.exceptions import NoSuchElementException, WebDriverException, TimeoutException
import sys

import pandas as pd
import numpy as np

seleniumマネジャーが利用可能かハージョン確認  
4.6以前であれば更新

In [ ]:
print(selenium.__version__)

## 詳細ページに遷移するための関数
* ~~WebdriverExceptionが発生した場合に待機して、リロードを行う~~<br>
* ~~2間隔でページ全体が更新されたか、最大3秒間確認する~~
* 3秒以上の動画広告に対処するため、この関数ではページ遷移と、エラー発生時のリロードのみ行う。<br>
5回リロードしても同じエラーが発生する場合には強制終了を行う

In [ ]:
def Safe_navigation(href, i=0):
  try:
    driver.get(href)
    # 例外オブジェクトを e としてキャッチ
  except Exception as e:
    if i >= 5:
      try:
        sys.exit()
      except SystemExit:
        print('ページ遷移不可のため、強制終了')
      raise
    else:
        print(f"Navigation error: {e}")
        i += 1
        Safe_navigation(href, i)

## 詳細ページからデータを取得する関数

In [ ]:
#絞込に従う、ランキングtop4~5店の詳細情報を取得する
def Store_detail_of_kind(kind, name_list, href_list, df_kind):
  # top4~50の店から店舗詳細情報を取得
  for i in range(len(href_list)):
    Safe_navigation(href_list[i])
    # 住所が登録されている要素を取得
    address = Get_address()
    # pointが登録されている要素を取得
    point = Get_point()
    # 緯度経度の要素を取得する
    latitude,  longitude = Get_map()
    # 駐車場の要素を取得。（classの登録がなかったため、テキスト部分で条件指定
    parking = Get_parking()
    # 定休日の要素を取得。（classの登録がなかったため、テキスト部分で条件指定）
    holiday = Get_holiday()
    Mk_detail_list_of_kind(kind, name_list[i], point, address, latitude, longitude, parking, holiday, df_kind)
  # 定期的なキャッシュの削除
  driver.get('chrome://settings/clearBrowserData')
  # ランキングのページに戻り、ジャンルのプルダウンを取得できるようにする
  driver.get('https://ramendb.supleks.jp/rank')
  # ページ全体がロードされるのを最大3秒間待機する
  WebDriverWait(driver, 3).until(
      lambda d: d.execute_script('return document.readyState') == 'complete'
  )

def Get_address():
  try:
    #addressの要素を2秒おきに確認しながら、最大3秒待機する
    address_ele = WebDriverWait(driver, 3, poll_frequency=2).until(
      EC.presence_of_element_located((By.XPATH, '//tbody/tr/td/span[@itemprop="address"]'))
    )
    address = address_ele.text
  # 住所が取得できなければ、Noneを取得
  except Exception as e:
    print(e)
    address = None
  return address
        
def Get_map():
  try:
    # mapの要素を確認しながら、最大3秒待機する。addressと同じページ内なので、確認秒数はデフォルトの500ミリ秒とする
    map_data_ele = WebDriverWait(driver, 3).until(
      EC.presence_of_element_located((By.ID, 'gpetitembedmap'))
    )
    map_data = map_data_ele.get_attribute('src')
    # srcの要素値を緯度経度だけ切り離す
    # q= という文字が入るので、スライスで取り除く
    map_data = map_data.split('&')[1][2:]
    # 緯度経度に分割する
    map_data = map_data.split(',')
    latitude = map_data[0]
    longitude = map_data[1]
  # 緯度経度が取得できなければ、Noneを取得
  except Exception as e:
    print(e)
    latitude = None
    longitude = None
  return latitude, longitude
        
def Get_parking():
  try:
    th_element = WebDriverWait(driver, 3).until(
      EC.presence_of_element_located((By.XPATH, '//tbody/tr/th[contains(text(), "駐車場")]'))
    )
    parking = th_element.find_element(By.XPATH, "./following-sibling::td").text
  except Exception as e:
    print(e)
    parking = None
  return parking
        
def Get_holiday():
  try:
    th_element = WebDriverWait(driver, 3).until(
      EC.presence_of_element_located((By.XPATH, '//tbody/tr/th[contains(text(), "定休日")]'))
    )
    holiday = th_element.find_element(By.XPATH, "./following-sibling::td").text
  except Exception as e:
    print(e)
    holiday = None
  return holiday

def Get_point():
  try:
    point_ele_int = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//span[@itemprop="ratingValue"]/span[@class="int"]'))
    )
    point_ele_float = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//span[@itemprop="ratingValue"]/span[@class="float"]'))
    )
    point = point_ele_int.text + point_ele_float.text
  # 住所が取得できなければ、Noneを取得
  except Exception as e:
    print(e)
    point = None
  return point

## 取得したデータをフレーム化する関数

In [ ]:
# 店舗名とポイント、店舗詳細を店舗ごとにリスト化する
def Mk_detail_list_of_kind(kind, name, point, address, latitude, longitude, parking, holiday, df_kind):
  detail_list = [kind]
  detail_list.append(name)
  detail_list.append(point)
  detail_list.append(address)
  detail_list.append(latitude)
  detail_list.append(longitude)
  detail_list.append(parking)
  detail_list.append(holiday)
  print(detail_list)
  df_kind.loc[len(df_kind)] = detail_list

## 分類ごとのランキングを取得する関数

In [ ]:
def Kind_cycle(df_kind, pager):
  for i in pager:
    # 分類別ごとのデータ取得
    # 分類別、スープ別のselect要素を取得する
    kinds_sel_ele = driver.find_element(By.XPATH, '//select[@name="style"]')
    # 分類別の要素をオブジェクトへ変換する
    kinds_sel_obj = Select(kinds_sel_ele)
    # オブジェクトの、value属性のリストを作成する
    kinds_val_list = [kind.get_attribute('value') for kind in kinds_sel_obj.options]
    kinds_text_list = [kind.text for kind in kinds_sel_obj.options]
    # 都道府県の要素をドロップダウンから選択して
    kinds_sel_obj.select_by_value(kinds_val_list[i])
    # 表示ボタンの要素を取得できるまで待機
    viewer = driver.find_element(By.XPATH, '//div[@class="left"]/input[@class="formbtn"]')
    # 要素（表示ボタン）が表示されるまでスクロールする
    driver.execute_script("arguments[0].scrollIntoView();", viewer)
    viewer.click()
    # 要素(top3の店名, top4~50の店名)がページ上に表示されるまで最大10秒待機、その後実行
    store_top3_ele = WebDriverWait(driver, 10, poll_frequency=2).until(
      EC.presence_of_all_elements_located((By.XPATH, '//div[@class="ranks"]/ul[@id="honor"]/li/div[@class="name"]/span/a'))
    )
    store_ele = WebDriverWait(driver, 10).until(
      EC.presence_of_all_elements_located((By.XPATH, '//table[@class="rank"]/tbody/tr/td/span[@class="name"]/a'))
    )
    # 取得したtop3の店名、詳細ページのurlをリスト形式で保存
    store_name_top3_list = [store_top3.text for store_top3 in store_top3_ele]
    href_list_top3 = [ hrefs.get_attribute('href') for hrefs in store_top3_ele]
    # 取得したtop4~50の店名、詳細ページのurlをリスト形式で保存
    store_name_list = [store.text for store in store_ele]
    href_list = [hrefs.get_attribute('href') for hrefs in store_ele]
    
    #top51~100が掲載されたページに遷移する
    driver.execute_script(f"window.location.href='/rank?page=2&style={kinds_val_list[i]}';")
    # 要素がページ上に表示されるまで最大5秒待機、その後実行
    store_ele_next_rank = WebDriverWait(driver, 10, poll_frequency=2).until(
      EC.presence_of_all_elements_located((By.XPATH, '//tbody/tr/td/span[@class="name"]/a'))
    )
    store_name_next_rank_list = [store.text for store in store_ele_next_rank]
    href_next_rank_list = [hrefs.get_attribute('href') for hrefs in store_ele_next_rank]
    
    # リストをそれぞれ結合する
    store_names_list = store_name_top3_list + store_name_list + store_name_next_rank_list
    hrefs_list = href_list_top3 + href_list + href_next_rank_list
    Store_detail_of_kind(kinds_text_list[i], store_names_list, hrefs_list, df_kind)

## 種類ごとのランキングを取得する

In [ ]:
# 新版
# 必要に応じてoptionsを設定
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# 暗黙的な待機
driver.implicitly_wait(20)
driver.get('https://ramendb.supleks.jp/rank')

# 実行するプルダウンのリスト
pager = [1,2,3,4,6,7,8,9,10,11,12,13,14,15,16]

# # データフレームの枠組みを作成する
columns_list_detail = ['kind', 'name', 'point', 'address', 'latitude', 'longitude', 'parking', 'holiday']
df_kind = pd.DataFrame(columns = columns_list_detail)
Kind_cycle(df_kind, pager)
df_kind.to_csv('../data/kinds_of_rank.csv',index=False)